In [ ]:
#Demo to show case stable diffusion using determined

## Before You Start: 🤗 Account, Access Token, and License

In order to use this repository's implementation of Stable Diffusion, you must:

* Have a [Huggingface account](https://huggingface.co/join).
* Have a [Huggingface User Access Token](https://huggingface.co/docs/hub/security-tokens).
* Accept the Stable Diffusion license (click on _Access
  repository_  [in this link)](https://huggingface.co/CompVis/stable-diffusion-v1-4).


## Training Setup

1. After including your user access token in the `const.yaml` config file by modifying the final part
of the lines which read

2. Update the resource pool name based on your cluster, this has been tested on 2 gpu slots of v100

```yaml
environment:
  environment_variables:
    - HF_AUTH_TOKEN=YOUR_HF_AUTH_TOKEN_HERE
resources:
  resource_pool: v100-gpu-pool #your GPU resource pool, tested against v100
```

## Fine-tuning
Fine-tuning experiment can be run by executing the following in the present directory

The below command will submit an experiment which introduces a new embedding vector into the world of Stable
Diffusion which we will train to correspond to the concept of the Determined AI logo, as represented
through
training images found in `/det_logos`


In [1]:
!det experiment create const.yaml .

Preparing files (/run/determined/workdir/determined-diffusion-sample) to send to master... 3.5MB and 22 files  
Created experiment 105


By default, sample images are generated during training which can be viewed by launching a
Tensorboard instance from the experiment in the WebUI.

## Prompt Inference

In [2]:
# Once the fine tuning is complete, grab the UUID of best checkpoint from the above experiment. pass the experiment id from above
!det experiment list-checkpoints --best 1 105

   Trial ID |   # of Batches | State     | Validation Metric   | UUID                                 | Resources                  | Size
------------+----------------+-----------+---------------------+--------------------------------------+----------------------------+---------
        685 |            250 | COMPLETED |                     | c502af07-f1d5-4476-8809-23aa5daddda8 | learned_embeddings_dict.pt | 289.6MB
            |                |           |                     |                                      | optimizer_state_dict.pt    |


In [6]:
# libraries for visualization and prompt inference
! pip install -qq jupyterlab-widgets==1.1.1 ipywidgets==7.7.2

In [27]:
checkpoint_uuids = ['c502af07-f1d5-4476-8809-23aa5daddda8'] #UUID of your best ckpt from the above run

# Create a textual inversion pipeline for prompt inference
from detsd import DetSDTextualInversionPipeline
detsd_pipeline = DetSDTextualInversionPipeline()

# load ckpts
detsd_pipeline.load_from_uuids(checkpoint_uuids)


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Building the pipeline...
--------------------------------------------------------------------------------
Done!
Successfully loaded checkpoints. All loaded concepts: ['<det-logo>']


In [28]:
# Create a directory in which to save the generated images:
!mkdir generated_images

In [29]:
# Grab the first concept which was loaded into the pipeline, if any, otherwise falling back to a default:
try:
    first_concept = detsd_pipeline.all_added_concepts[0]
except IndexError:
    first_concept = 'orange brain'
print(f'Using "{first_concept}" as first_concept in the below')
print(f'All available concepts: {detsd_pipeline.all_added_concepts}')

Using "<det-logo>" as first_concept in the below
All available concepts: ['<det-logo>']


In [ ]:
# Generate and inspect images based on prompt. Generating right prompts is an art
prompt = f"a watercolor painting on textured paper of a {first_concept} using soft strokes, pastel colors, incredible composition, masterpiece"
imgs = detsd_pipeline(prompt=prompt,
                      parallelize_factor=2,
                      rows=2,
                      cols=2,
                      num_inference_steps=50,
                      seed=2147483647,
                      guidance_scale=7.5,
                      saved_img_dir='generated_images')
imgs